# *_FRIENDSTER - Social Network_*

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Row
import re

In [2]:
sc.getConf().getAll()

[('spark.repl.class.uri', 'spark://192.168.254.21:45603/classes'), ('spark.driver.port', '45603'), ('spark.repl.class.outputDir', '/tmp/spark-a21ea0a8-383b-4b02-803d-580e77b19be2/repl-4ef1050e-cd21-4108-a86e-6b8abcd9660f'), ('spark.jars', 'file:/usr/local/share/jupyter/kernels/apache_toree_pyspark/lib/toree-assembly-0.2.0.dev1-incubating-SNAPSHOT.jar'), ('spark.app.id', 'local-1531902303212'), ('spark.master', 'local[*]'), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.app.name', 'Apache Toree'), ('spark.driver.host', '192.168.254.21')]

In [3]:
conf = SparkConf().setAll([('spark.app.name', 'Friendster-mapreduce-by-key'),("spark.dynamicAllocation.enabled", "true"),("spark.dynamicAllocation.minExecutors", 1), ("spark.dynamicAllocation.maxExecutors", 5), ("spark.shuffle.service.enabled", "true") ])
sc.stop()
sc = SparkContext(conf=conf)

In [4]:
sc.getConf().getAll()

[('spark.repl.class.uri', 'spark://192.168.254.21:35408/classes'), ('spark.driver.port', '35408'), ('spark.dynamicAllocation.minExecutors', '1'), ('spark.shuffle.service.enabled', 'true'), ('spark.jars', 'file:/usr/local/share/jupyter/kernels/apache_toree_pyspark/lib/toree-assembly-0.2.0.dev1-incubating-SNAPSHOT.jar'), ('spark.executor.id', 'driver'), ('spark.dynamicAllocation.maxExecutors', '5'), ('spark.app.name', 'Friendster-mapreduce-by-key'), ('spark.repl.class.outputDir', '/tmp/spark-a21ea0a8-383b-4b02-803d-580e77b19be2/repl-4ef1050e-cd21-4108-a86e-6b8abcd9660f'), ('spark.app.id', 'local-1531902309450'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.dynamicAllocation.enabled', 'true'), ('spark.driver.host', '192.168.254.21')]

In [5]:
sc

<SparkContext master=local[*] appName=Friendster-mapreduce-by-key>

In [6]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

# Création du graphe 

> **Hypothèse : ** Les liens entre les utilisateurs sont des liens 
    orientés entre utilisateur influant et le suiveur
    

In [7]:
import org.apache.spark.graphx.GraphLoader
//quelques aretes sont définies dans les 2 sens 
val users = (sc.textFile("/home/pb19121/datagraphx/WITH_ATT_NODES.csv")
  .map(line => line.split(",")).map( x => (x.head.toLong, x.tail)))

In [8]:
// Parse the edge data which is already in userId -> userId format
val followerGraph = GraphLoader.edgeListFile(sc, "/home/pb19121/datagraphx/friendsterfollowers.txt")

In [9]:
//# Attach the user attributes
val graph = followerGraph.outerJoinVertices(users) {
  case (uid, deg, Some(attrList)) => attrList
 // # Some users may not have attributes so we set them as empty
  case (uid, deg, None) => Array.empty[String]
}

In [10]:
//Ajouter les attributs à partir des calculs de degres, indegree et outdegree dans le fichiers des users
import org.apache.spark.graphx.GraphLoader

val users = (sc.textFile("/home/pb19121/datagraphx/friendsterusers.txt")
  .map(line => line.split(",")).map( parts => (parts.head.toLong, parts.tail) ))

// Parse the edge data which is already in userId -> userId format
val edgesGraph = GraphLoader.edgeListFile(sc, "/home/pb19121/datagraphx/friendsterfollowers.txt")

//# Attach the user attributes
val graph = edgesGraph.outerJoinVertices(users) {
  case (uid, deg, Some(attrList)) => attrList
 // # Some users may not have attributes so we set them as empty
  case (uid, deg, None) => Array.empty[String]
}

In [11]:
graph

org.apache.spark.graphx.impl.GraphImpl@7c8cdd50

In [12]:
// Restrict the graph to users with usernames and names
val subgraph = graph.subgraph(vpred = (vid, attr) => attr.size == 2)


// Compute the PageRank
val pagerankGraph = subgraph.pageRank(0.001)

// Get the attributes of the top pagerank users
val userInfoWithPageRank = subgraph.outerJoinVertices(pagerankGraph.vertices) {
  case (uid, attrList, Some(pr)) => (pr, attrList.toList)
  case (uid, attrList, None) => (0.0, attrList.toList)
}

In [13]:
println(userInfoWithPageRank.vertices.top(10)(Ordering.by(_._2._1)).mkString("\n"))

(1,(8019.989012755636,List(N1, n1)))
(2,(3473.697419631351,List(N2, n2)))
(82,(1691.8315267036721,List(N82, n82)))
(11,(1038.692904959271,List(N11, n11)))
(3,(1033.6143079036847,List(N3, n3)))
(13,(920.921867922792,List(N13, n13)))
(4,(670.6766422815854,List(N4, n4)))
(54,(648.4665619563988,List(N54, n54)))
(5,(546.9728220426837,List(N5, n5)))
(20,(540.7584562672422,List(N20, n20)))


In [14]:
// Get the degree of users
val userInfoWithDegree = subgraph.outerJoinVertices(userInfoWithPageRank.degrees) {
  case (uid,attrList,   Some(y)) => ( y)
  case (uid,attrList,  None) => (0)
}

In [15]:
// Get the degree of users
println(userInfoWithDegree.vertices.top(10)(Ordering.by(_._2)).mkString("\n"))

(3687,621)
(3552,592)
(3570,592)
(3553,548)
(82,517)
(3542,505)
(212,502)
(3674,446)
(2680,439)
(3651,428)


# Distribution de degrés

In [17]:
// Define a reduce operation to compute the highest degree vertex
def max(a: (VertexId, Int), b: (VertexId, Int)): (VertexId, Int) = {
  if (a._2 > b._2) a else b
}
// Compute the max degrees
val maxInDegree: (VertexId, Int)  = graph.inDegrees.reduce(max)
val maxOutDegree: (VertexId, Int) = graph.outDegrees.reduce(max)
val maxDegrees: (VertexId, Int)   = graph.degrees.reduce(max)

In [18]:
print (" max indegree : ")
maxInDegree

 max indegree : 

(3687,620)

In [19]:
print (" max outdegree : ")
maxOutDegree

 max outdegree : 

(111058,165)

In [20]:
print (" max degree : ")
maxDegrees

 max degree : 

(3687,621)

In [21]:
val degrees: VertexRDD[Int] = graph.degrees

In [22]:
degrees.take(5).foreach(println)

(108150,3)
(129434,5)
(91902,16)
(68522,14)
(23776,4)


In [23]:
val indegrees: VertexRDD[Int] = graph.inDegrees

In [24]:
val outdegrees: VertexRDD[Int] = graph.outDegrees

In [25]:
degrees.map(_._1).stats()

(count: 100199, mean: 69079.757972, stdev: 41109.577764, max: 151837.000000, min: 1.000000)

In [26]:
indegrees.map(_._1).stats()

(count: 79262, mean: 63332.714163, stdev: 39503.759873, max: 151820.000000, min: 1.000000)

In [27]:
outdegrees.map(_._1).stats()

(count: 100198, mean: 69080.447394, stdev: 41109.203660, max: 151837.000000, min: 2.000000)

In [28]:
degrees.map(_._2).stats()

(count: 100199, mean: 19.599397, stdev: 24.609730, max: 621.000000, min: 1.000000)

In [29]:
indegrees.map(_._2).stats()

(count: 79262, mean: 12.388282, stdev: 20.920199, max: 620.000000, min: 1.000000)

In [30]:
outdegrees.map(_._2).stats()

(count: 100198, mean: 9.799796, stdev: 11.073388, max: 165.000000, min: 1.000000)

# Coeficient de clustering : 

In [31]:
val triplets = graph.triplets

In [32]:
triplets.count()

981920

In [33]:
triplets.take(5).foreach(println)

((2,[Ljava.lang.String;@1ca2e0ca),(1,[Ljava.lang.String;@521260d0),1)
((3,[Ljava.lang.String;@412d7f4d),(1,[Ljava.lang.String;@521260d0),1)
((4,[Ljava.lang.String;@5045ee6a),(1,[Ljava.lang.String;@521260d0),1)
((4,[Ljava.lang.String;@5045ee6a),(2,[Ljava.lang.String;@1ca2e0ca),1)
((5,[Ljava.lang.String;@5b6ae018),(1,[Ljava.lang.String;@521260d0),1)


In [34]:
val triCounts = graph.triangleCount()

In [35]:
triCounts.vertices.map(x => x._2).stats()

(count: 100199, mean: 63.479735, stdev: 140.725195, max: 4142.000000, min: 0.000000)

In [36]:
val maxTrisGraph = triCounts.degrees.mapValues(d => d * (d - 1) / 2.0)

In [37]:
maxTrisGraph

VertexRDDImpl[555] at RDD at VertexRDD.scala:57

In [38]:
val clusterCoefGraph = triCounts.vertices.innerJoin(maxTrisGraph) {
  (vertexId, triCount, maxTris) => {
    if (maxTris == 0) 0
    else triCount / maxTris
  }
}

In [39]:
clusterCoefGraph.map(_._2).sum() / triCounts.vertices.count()

0.2672521165006904

> **Notes : **
- on obtient 26% caractéristique des réseaux de petits mondes



In [40]:
val connectedComponentGraph: Graph[VertexId, Int] = graph.connectedComponents()

In [41]:
def sortedConnectedComponents(connectedComponents: Graph[VertexId, _]): Seq[(VertexId, Long)] = {
  val componentCounts = connectedComponents.vertices.map(_._2).countByValue
  componentCounts.toSeq.sortBy(_._2).reverse
}

In [42]:
val componentCounts = sortedConnectedComponents( connectedComponentGraph)

In [43]:
componentCounts.size

1

In [44]:
componentCounts.take(10).foreach(println)

(1,100199)
